In [ ]:
# Importing dependencies
# OS to navigate through directories and create folders
import os
# Usual scientific packages
import numpy as np
import matplotlib.pyplot as plt
# Library to read ND2 format
from nd2reader import ND2Reader
# Libraries to work with and save images
import cv2
import tiffile
# Libraries to get an estimate of completion / time to required to extract images
import time
from tqdm.notebook import tqdm

We intend to segment condensates located on different z-levels across multiple XY points. On top of this, condensates might change in their equatorial plane across timeframes. To reduce background out-of-plane fluorescence signal, we employ a quick autofocusing algorithm found in literature, based on normalised variance.  
Refs:    
https://onlinelibrary.wiley.com/doi/abs/10.1002/cyto.990060202 - REF for normalised variance algorithm    
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2965442/#R7 - Review

In [ ]:
def norm_variance(img): 
    return (np.std(img, ddof = 0))**2/np.mean(img)

In [ ]:
# Absolute path for results directory - change `/ABSOLUTE/PATH/TO/RESULTS/` with relevant absolute path
results_dir = "/ABSOLUTE/PATH/TO/RESULTS/"
# Folder in which original raw ND2 data is stored - change `/ABSOLUTE/PATH/TO/FILE/` with relevant absolute path
file_dir = "/ABSOLUTE/PATH/TO/FILE/"
filename = "timelapse.nd2"
filepath = file_dir + filename

In [ ]:
## Example with single channel extraction (for A and B condensates which are fluorescent in a single channel)
# Initialise Reader
reader = ND2Reader(filepath)

# Extract information on number of timeframes and z-planes/levels
# If needed, zlevels can also be restricted to a narrower range after manual inspection of the z-stacks
timeframes, zlevels = reader.metadata['frames'], reader.metadata['z_levels']

# Select the XY points to extract (here the first XY points for A and B condensates in bulk)
points = [
    6, # 7, 8, 
    9, # 10, 11, 
    # 12, 13, 14
]
# Assign experiment name for results to be saved
experiment_name = "extraction_samples_XYZ"

# Set flag to decide whether normalised variance profiles vs z are plotted or not
plot_flag = False

# Move to results directory
os.chdir(results_dir)
# Create new sub-dir to store analysis results for this particular experiment
path = results_dir + experiment_name + "/"
os.makedirs(path, exist_ok = True)
print("Directory created successfully")
os.chdir(path)

# Loop through XY points
for point, i in zip(points, tqdm(range(len(points)))):
    # Assign a readable and meaningful sample name == reflecting sample content
    if point in [6, 7, 8]: 
        name = "A_bulk"
        # Restricting analysis of channels to MG
        channels = {0: 'MG', 1:'DFHBI'}
        channel_flags = {0 : True, 1: False}  # only MG (0) is used
    elif point in [9, 10, 11]: 
        name = "B_bulk"
        # Restricting analysis of channels to DFHBI
        channels = {1:'DFHBI', 0: 'MG'}
        channel_flags = {0 : False, 1: True}  # only DFHBI (1) is used
    print('Point: ', point, ' containing sample: ', name)
    
    for channel, ch in zip(channels.keys(), tqdm(range(len(channels)))):
        if channel_flags[channel] == True:
            print('Channel: ', channels[channel])
            best_focus_planes = []
            # Loop through timepoints
            for timepoint in tqdm(timeframes): 
                nvs = np.array(
                    [
                        norm_variance(
                            np.array(
                                reader.get_frame_2D(c = channel, t = timepoint, z = z_plane, v = point),
                                dtype = np.uint16
                            )
                        )
                        for z_plane in zlevels
                    ]
                )
                best_focus_plane = np.argmax(nvs)
                if plot_flag: 
                    plt.figure(figsize = (2, 2))
                    plt.plot(zlevels, nvs)
                    plt.axvline(x = best_focus_plane)
                    plt.xlabel('Z-Plane')
                    plt.ylabel('Normalised Variance')
                    plt.show()
                best_focus_planes.append(best_focus_plane)
                
            # Correct planes of focus in early timepoints -- if there is no signal, usually the found optimal z-plane
            # is higher in the bulk rather than at the bottom, near the coverslip, where it should be. 
            # However, we know the signal will be at the bottom -> we can reposition the optimal plane
            # to match the first reasonable one in the timeframe sequence.
            # In practice, if this algorithm finds the optimal plane above 80% of the scanned z-volume, we reposition it
            # to the first occurrence of a z-plane below such threshold.
            print(f"Preliminary analysis completed. These are the planes of best focus we found: {best_focus_planes}")
            indices = []
            for focus_plane, ind in zip(best_focus_planes, range(len(best_focus_planes))): 
                if focus_plane > int(0.80*zlevels[-1]): 
                    indices.append(ind)
            if indices != []: 
                first_good_index = np.max(indices) + 1
                for ind in indices: 
                    best_focus_planes[ind] = best_focus_planes[first_good_index]
                print(f"Had to correct some planes - here is a revision: {best_focus_planes}")
        
        print("Moving on to extraction now...")
        image_stack = []
        for timepoint in tqdm(timeframes):
            planes = []
            # the range below can be tweaked to perform a max z-projection by increasing the number of z-planes
            for z_plane in range(best_focus_planes[timepoint], best_focus_planes[timepoint]+1): 
                planes.append(np.array(reader.get_frame_2D(c = channel, t = timepoint, z = z_plane, v = point), dtype = np.uint16))
            # in this case this operation does nothing as there is a single z-plane
            image_stack.append(np.max(np.array(planes), axis = 0)) 
        image_stack = np.array(image_stack)
        im_file_name = channels[channel] + "_" + name + "_" + str(point%3) + ".ome.tif"
        print("Timelapse will now be saved as: ", im_file_name)
        tiffile.imwrite(im_file_name, image_stack, metadata={'axes': 'TYX'}, compression ='zlib')


In [ ]:
## Example with coupled channel extraction (for C condensates which are fluorescent in both MG and DFHBI channels)
# Initialise Reader
reader = ND2Reader(filepath)

# Extract information on number of timeframes and z-planes/levels
timeframes = reader.metadata['frames']
zlevels = reader.metadata['z_levels']
# Select the XY points to extract (here the first XY point for C in bulk)
points = [
    # 6, 7, 8, 
    # 9, 10, 11, 
    12, # 13, 14
]
# Assign experiment name for results to be saved
experiment_name = "extraction_samples_XYZ"

# Set flag to decide whether normalised variance profiles vs z are plotted or not
plot_flag = True


# Move to results directory
os.chdir(results_dir)
# Create new sub-dir to store analysis results for this particular experiment
path = results_dir + experiment_name + "/"
os.makedirs(path, exist_ok = True)
print("Directory created successfully")
os.chdir(path)

# Loop through XY points
for point, i in zip(points, tqdm(range(len(points)))):
    # Assign a readable and meaningful sample name == reflecting sample content
    if point in [12, 13, 14]: 
        name = "C_bulk"
        # Both channels for C
        channels = {0: 'MG', 1:'DFHBI'}
        channel_flags = {0 : True, 1: True}
    print('Point: ', point, ' containing sample: ', name)
    
    best_focus_planes = []
    # Loop through timepoints
    for timepoint in tqdm(timeframes): 
        # for MG channel
        nvs_ch0 = np.array(
            [
                norm_variance(
                    np.array(
                        reader.get_frame_2D(c = 0, t = timepoint, z = z_plane, v = point),
                        dtype = np.uint16
                    )
                )
                for z_plane in zlevels
            ]
        )
        # for DFHBI channel
        nvs_ch1 = np.array(
            [
                norm_variance(
                    np.array(
                        reader.get_frame_2D(c = 1, t = timepoint, z = z_plane, v = point),
                        dtype = np.uint16
                    )
                )
                for z_plane in zlevels
            ]
        )
        # Sum channel contributions and find max normalised variance z-plane
        nvs = nvs_ch0 + nvs_ch1
        best_focus_plane = np.argmax(nvs)
        if plot_flag: 
            plt.subplots(1, 3, figsize = (7, 2))
            plt.subplot(131)
            plt.plot(zlevels, nvs_ch0)
            plt.xlabel('Z-Plane')
            plt.ylabel('Normalised Variance')
            plt.title('Ch0')
            plt.subplot(132)
            plt.xlabel('Z-Plane')
            plt.plot(zlevels, nvs_ch1)
            plt.title('Ch1')
            plt.subplot(133)
            plt.plot(zlevels, nvs)
            plt.axvline(x = best_focus_plane)
            plt.xlabel('Z-Plane')
            plt.title('Sum')
            plt.show()
        best_focus_planes.append(best_focus_plane)

    # Correct planes of focus in early timepoints -- if there is no signal, usually the found optimal z-plane
    # is higher in the bulk rather than at the bottom, near the coverslip, where it should be. 
    # However, we know the signal will be at the bottom -> we can reposition the optimal plane
    # to match the first reasonable one in the timeframe sequence.
    # In practice, if this algorithm finds the optimal plane above 80% of the scanned z-volume, we reposition it
    # to the first occurrence of a z-plane below such threshold.
    print(f"Preliminary analysis completed. These are the planes of best focus we found: {best_focus_planes}")
    indices = []
    for focus_plane, ind in zip(best_focus_planes, range(len(best_focus_planes))): 
        if focus_plane > int(0.80*zlevels[-1]): 
            indices.append(ind)
    if indices != []: 
        first_good_index = np.max(indices) + 1
        for ind in indices: 
            best_focus_planes[ind] = best_focus_planes[first_good_index]
        print(f"Had to correct some planes - here is a revision: {best_focus_planes}")

    print("Moving on to extraction now...")
    for channel, ch in zip(channels.keys(), tqdm(range(len(channels)))):
        print(channel)
        image_stack = []
        for timepoint in tqdm(timeframes):
            planes = []
            # the range below can be tweaked to perform a max z-projection by increasing the number of z-planes
            for z_plane in range(best_focus_planes[timepoint], best_focus_planes[timepoint]+1): 
                planes.append(np.array(reader.get_frame_2D(c = channel, t = timepoint, z = z_plane, v = point), dtype = np.uint16))
            # in this case this operation does nothing as there is a single z-plane
            image_stack.append(np.max(np.array(planes), axis = 0))
        image_stack = np.array(image_stack)
        im_file_name = channels[channel] + "_" + name + "_" + str(point%3) + ".ome.tif"
        print("Timelapse will now be saved as: ", im_file_name)
        tiffile.imwrite(im_file_name, image_stack, metadata={'axes': 'TYX'}, compression ='zlib')